In [2]:

import os, sys



from config import DATABASE_URI
from models import Earnings_release, Security, Base
from scrapping_sources.Macrotrend import Macrotrend
# from earnings_release.earnings_release import last_period_db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import pandas as pd
import datetime

DAY_VARIABLE = 30
look_back_date = str(datetime.date.today() - datetime.timedelta(days=DAY_VARIABLE))

engine = create_engine(DATABASE_URI)
Session = sessionmaker(bind=engine)
s = Session()

querry = s.query(Security.ticker, Security.id).all()
security_map = {querry[i][0]: querry[i][1] for i, v in zip(range(len(querry)), range(len(querry)))}

statements = [
    'income_statement',
    'balance_sheet',
    'cash_flow_statement',
    'financial_ratios'
]
time_format = [
    'annual',
    'quarterly'
]

M = Macrotrend()

def convert_to_ending_period_format(date):
    """Returns """
    result = datetime.datetime.strptime(str(date), '%Y-%m-%d').strftime("%Y-%m")
    return result

def last_period_db(ticker):
    try:
        r = s.query(Base.metadata.tables['income_statement_quarterly'].columns['date']).where(
            Base.metadata.tables['income_statement_quarterly'].columns['ticker'] == str(ticker)
        ).all()
        r.sort(reverse=True)
        r =  str(r[0][0])
        return convert_to_ending_period_format(r)
    except:
        return None

def fetch_all_statement(ticker, stmnt, time_format):

    r = s.query(Base.metadata.tables[f'{stmnt}_{time_format}']).where(\
            Base.metadata.tables[f'{stmnt}_{time_format}'].columns['ticker'] ==str(ticker)).all()
    return r

def statement_table_log(ticker, statement, time_format, status):

    try:
        security_id= security_map[f"{ticker}"]
        load = pd.DataFrame(
        [
            {
                "date"          : str(datetime.date.today()),
                "ticker"        : f"{ticker}",
                "security_id"   : security_id,
                "statement"     : f"{statement}",
                "time_format"   : time_format,
                "status"        : status
              }
        ]
    )

        load.to_sql(con=engine, name="statements_table_log", if_exists="append", index=False)
    except:
        security_id = None

    load = pd.DataFrame(
        [
            {
                "date"          : str(datetime.date.today()),
                "ticker"        : f"{ticker}",
                "security_id"   : security_id,
                "statement"     : f"{statement}",
                "time_format"   : time_format,
                "status"        : status
              }
        ]
    )

    load.to_sql(con=engine, name="statements_table_log", if_exists="append", index=False)

def update_db(ticker, stmnt, t_format):

    latest = M.arrange_data(ticker, stmnt, t_format)
    print(latest)
    in_database = pd.DataFrame(fetch_all_statement(ticker, stmnt, t_format))

    try:
        in_database.columns = ['date','statement','ticker','line_item', 'amount']
        in_database['security_id'] = in_database.ticker.map(security_map)
        in_database = M.move_column(in_database, 'security_id', 3)
        latest['security_id'] = latest.ticker.map(security_map)
        convert_dict = {
            'date': str,
            'statement': str,
            'ticker': str,
            'security_id': int,
            'line_item': str,
            'amount': float
        }

        in_database = in_database.astype(convert_dict)
        latest = latest.astype(convert_dict)

    except:

        print("Something went wrong in the dataframe creation")

    if isinstance(in_database, pd.DataFrame) and isinstance(latest, pd.DataFrame):

        if latest.shape[0] == in_database.shape[0]:

            print(f"Security {ticker} is already up to date")

        else:

            in_database['date'] = pd.to_datetime(in_database.date)
            indb_date_set = set(in_database.date.values)
            latest['date'] = pd.to_datetime(latest.date)
            update_date_set = set(latest.date.values)
            update = latest[latest.date.isin(list(update_date_set - indb_date_set))]
            # UPDATE THE CORRESPONDING TABLE
            update.to_sql(con = engine, name=f"{stmnt.replace('-','_')}_{time_format}", if_exists='append', index=False)
            # UPDATE THE STATEMENTS TABLE LOG
            statement_table_log(ticker, stmnt, t_format, status="updated")
            
    elif not isinstance(in_database, pd.DataFrame):

        statement_table_log(
            ticker,
            stmnt, 
            t_format, 
            status=f"{ticker} Not in Database"
        )

    else:

        statement_table_log(
            ticker,
            stmnt, 
            t_format, 
            status=f"{ticker} no data availble on M"
        )

In [3]:
update_db('CANF', 'balance_sheet', "quarterly")

CANF <Response [404]>
No data available for CANF, balance_sheet, quarterly
None
Something went wrong in the dataframe creation


In [6]:
M.arrange_data('CANF', 'cash-flow-statement', "quarterly")

CANF <Response [200]>


,date,statement,ticker,security_id,line_item,amount
0,2021-06-30,CF-Q,CANF,None,Total Depreciation And Amortization - Cash Flow,0.008
1,2021-06-30,CF-Q,CANF,None,Other Non-Cash Items,0.038
2,2021-06-30,CF-Q,CANF,None,Total Non-Cash Items,0.046
3,2021-06-30,CF-Q,CANF,None,Change In Accounts Receivable,-0.688
4,2021-06-30,CF-Q,CANF,None,Change In Inventories,0.0
...,...,...,...,...,...,...
695,2011-12-31,CF-Q,CANF,None,Financial Activities - Other,3.63
696,2011-12-31,CF-Q,CANF,None,Cash Flow From Financial Activities,4.95
697,2011-12-31,CF-Q,CANF,None,Net Cash Flow,-0.8
698,2011-12-31,CF-Q,CANF,None,Stock-Based Compensation,0.0
